# MPI hello world:

In [1]:
%%writefile src/mpi_hello.c
#include<mpi.h>
#include<stdio.h>

int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    if (rank!=0)
        printf("Hello I'm %d of %d\n", rank, size);
    else
        printf("I'm master of %d!!!\n", size);
    MPI_Finalize();
    return 0;
}

Overwriting src/mpi_hello.c


In [2]:
!mpicc -pedantic -Wall -o bin/mpi_hello src/mpi_hello.c
!mpiexec -n 4 bin/mpi_hello

I'm master of 4!!!
Hello I'm 3 of 4
Hello I'm 1 of 4
Hello I'm 2 of 4


# MPI blocking send message in loop

In [3]:
%%writefile src/mpi_send.c
#include<mpi.h>
#include<stdio.h>

int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int i;
    int message=rank;
    int rec_buf;
    int count=1;
    int tag=0;
    MPI_Status status;
    double starttime, endtime;
    
    MPI_Barrier(MPI_COMM_WORLD);
    starttime=MPI_Wtime();
    for(i=0;i<10;i++){
        if (rank%2==0){
            MPI_Send(&message, count, MPI_INT, (rank+1)%size, tag, MPI_COMM_WORLD);
            MPI_Recv(&rec_buf, count, MPI_INT, (rank-1)%size, tag, MPI_COMM_WORLD,&status);
        }
        else{
            MPI_Recv(&rec_buf, count, MPI_INT, (rank-1)%size, tag, MPI_COMM_WORLD,&status);
            MPI_Send(&message, count, MPI_INT, (rank+1)%size, tag, MPI_COMM_WORLD);
        }
        printf("rank %d received message %d\n", rank, rec_buf);
        message=rec_buf;
        MPI_Barrier(MPI_COMM_WORLD);
        if (rank==0) printf("\n");
        MPI_Barrier(MPI_COMM_WORLD);
    }
    MPI_Barrier(MPI_COMM_WORLD);
    endtime=MPI_Wtime();
    
//Fun with datatypes
    if (rank==0){
        printf("overall time: %f\n", endtime-starttime);
        int intsize;
        int charsize;
        MPI_Type_size(MPI_INT, &intsize);
        MPI_Type_size(MPI_CHAR, &charsize);
        
        printf("Size of Int: %d\nSize of Char: %d\n", intsize, charsize);
        MPI_Get_count(&status, MPI_CHAR, &count); //will be 4, as int is 4 bytes and char is 1 byte
        printf("count was %d\n",count);
    }
    
    
    MPI_Finalize();
    return 0;
}

Overwriting src/mpi_send.c


In [4]:
!mpicc -pedantic -Wall -o bin/mpi_send src/mpi_send.c
!mpiexec -n 4 bin/mpi_send

rank 0 received message 3
rank 1 received message 0
rank 2 received message 1
rank 3 received message 2

rank 0 received message 2

rank 0 received message 1

rank 1 received message 3
rank 1 received message 2
rank 1 received message 1
rank 2 received message 0
rank 2 received message 3
rank 2 received message 2
rank 3 received message 1
rank 3 received message 0
rank 3 received message 3
rank 2 received message 1
rank 0 received message 0

rank 0 received message 3

rank 0 received message 2

rank 1 received message 0
rank 1 received message 3
rank 1 received message 2
rank 3 received message 2
rank 3 received message 1
rank 3 received message 0
rank 3 received message 3
rank 1 received message 1
rank 2 received message 0
rank 2 received message 3
rank 2 received message 2
rank 0 received message 1

rank 0 received message 0

rank 0 received message 3
rank 3 received message 2
rank 3 received message 1
rank 1 received message 0
rank 1 received message 3
rank 2 received message 1
rank

# MPI collective
## MPI_Bcast:

In [5]:
%%writefile src/mpi_bcast.c
#include<mpi.h>
#include<stdio.h>

int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int message;
    int count=1;
    int root=0;
    
    printf("uninitialized message: %d\n",message);
    MPI_Barrier(MPI_COMM_WORLD);
    if (rank==0)
        message=31415;
    MPI_Bcast(&message, count, MPI_INT, root, MPI_COMM_WORLD);
    printf("rank: %d, message: %d\n",rank,message);

    
    MPI_Finalize();
    return 0;
}

Overwriting src/mpi_bcast.c


In [6]:
!mpicc -pedantic -Wall -o bin/mpi_bcast src/mpi_bcast.c
!mpiexec -n 4 bin/mpi_bcast

uninitialized message: 21880
uninitialized message: 21998
uninitialized message: 22045
uninitialized message: 21882
rank: 0, message: 31415
rank: 1, message: 31415
rank: 2, message: 31415
rank: 3, message: 31415


## MPI_Reduce:
### result only available on root

In [7]:
%%writefile src/mpi_reduce.c
#include<mpi.h>
#include<stdio.h>

int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int message;
    int count=1;
    int root=0;
    
    printf("uninitialized message: %d\n",message);
    MPI_Barrier(MPI_COMM_WORLD);
    MPI_Reduce(&rank, &message, count, MPI_INT, MPI_SUM, root, MPI_COMM_WORLD);
    printf("rank: %d, message: %d\n",rank,message);

    MPI_Finalize();
    return 0;
}

Overwriting src/mpi_reduce.c


In [8]:
!mpicc -pedantic -Wall -o bin/mpi_reduce src/mpi_reduce.c
!mpiexec -n 4 bin/mpi_reduce

uninitialized message: 21971
uninitialized message: 22067
uninitialized message: 21966
uninitialized message: 21991
rank: 2, message: 21991
rank: 3, message: 21966
rank: 1, message: 21971
rank: 0, message: 6


## MPI_Allreduce

In [9]:
%%writefile src/mpi_allreduce.c
#include<mpi.h>
#include<stdio.h>

int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int message;
    int count=1;
    
    printf("uninitialized message: %d\n",message);
    MPI_Barrier(MPI_COMM_WORLD);
    MPI_Allreduce(&rank, &message, count, MPI_INT, MPI_SUM, MPI_COMM_WORLD);
    printf("rank: %d, message: %d\n",rank,message);

    MPI_Finalize();
    return 0;
}

Overwriting src/mpi_allreduce.c


In [10]:
!mpicc -pedantic -Wall -o bin/mpi_allreduce src/mpi_allreduce.c
!mpiexec -n 4 bin/mpi_allreduce

uninitialized message: -767154400
uninitialized message: -701385664
uninitialized message: -97099616
uninitialized message: -1491143888
rank: 1, message: 6
rank: 3, message: 6
rank: 2, message: 6
rank: 0, message: 6


## Gather

In [11]:
%%writefile src/gather_int.c
#include<mpi.h>
#include<stdio.h>
#include<string.h>
#include<stdlib.h>
int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int sendbuf=rank;
    int sendcount=1;
    int recv_data[size];
    int recv_count=sendcount;
    int root=0;
    MPI_Gather(&sendbuf, sendcount, MPI_INT, recv_data, recv_count, MPI_INT, root, MPI_COMM_WORLD);
    printf("rank %d,sendbuf %d, recv_data %d%d%d%d\n",rank,sendbuf,recv_data[0],recv_data[1],recv_data[2],recv_data[3]);

    MPI_Finalize();
    return 0;
}

Overwriting src/gather_int.c


In [12]:
!mpicc -pedantic -Wall -o bin/gather_int src/gather_int.c
!mpiexec -n 4 bin/gather_int

rank 1,sendbuf 1, recv_data 123974505632766-36831180921846
rank 3,sendbuf 3, recv_data 82972059232767-161231616121940
rank 0,sendbuf 0, recv_data 0123
rank 2,sendbuf 2, recv_data -161947398432766102271027122066


In [13]:
%%writefile src/gather_char.c
#include<mpi.h>
#include<stdio.h>
#include<string.h>
#include<stdlib.h>
int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    char sendbuf[3]="W A";
    int sendcount=3;
    char recv_data[strlen(sendbuf)*size+1];
    int recv_count=sendcount;
    int root=0;
       
    MPI_Gather(sendbuf, sendcount, MPI_CHAR, recv_data, recv_count, MPI_CHAR, root, MPI_COMM_WORLD);
    recv_data[strlen(sendbuf)*size]='\0';
    printf("rank %d, sendbuf %s, recv_data %s\n",rank,sendbuf,recv_data);

    MPI_Finalize();
    return 0;
}

Overwriting src/gather_char.c


In [14]:
!mpicc -pedantic -Wall -o bin/gather_char src/gather_char.c
!mpiexec -n 4 bin/gather_char

rank 3, sendbuf W A, recv_data 
rank 1, sendbuf W A, recv_data 
rank 0, sendbuf W A, recv_data W AW AW AW A
rank 2, sendbuf W A, recv_data 


In [15]:
%%writefile src/gather_str.c
#include<mpi.h>
#include<stdio.h>
#include<string.h>
#include<stdlib.h>
int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int sendcount=1;
    char sendbuf[sendcount];
    snprintf(sendbuf, sendcount+1, "%d", rank+4);

    char recv_data[size];
    int recv_count=sendcount;
    int root=0;
    MPI_Gather(sendbuf, sendcount, MPI_CHAR, recv_data, recv_count, MPI_CHAR, root, MPI_COMM_WORLD);
    printf("rank %d, sendbuf %s, recv_char %s\n",rank,sendbuf,recv_data);

    MPI_Finalize();
    return 0;
}

Overwriting src/gather_str.c


In [16]:
!mpicc -pedantic -Wall -o bin/gather_str src/gather_str.c
!mpiexec -n 4 bin/gather_str

rank 3, sendbuf 7, recv_char 
rank 1, sendbuf 5, recv_char 
rank 0, sendbuf 4, recv_char 4567
rank 2, sendbuf 6, recv_char 


## Allgather

In [17]:
%%writefile src/allgather_int.c
#include<mpi.h>
#include<stdio.h>
#include<string.h>
#include<stdlib.h>
int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int sendbuf=rank;
    int sendcount=1;
    int recv_data[size];
    int recv_count=sendcount;
    MPI_Allgather(&sendbuf, sendcount, MPI_INT, recv_data, recv_count, MPI_INT, MPI_COMM_WORLD);
    printf("rank %d,sendbuf %d, recv_data %d%d%d%d\n",rank,sendbuf,recv_data[0],recv_data[1],recv_data[2],recv_data[3]);

    MPI_Finalize();
    return 0;
}

Overwriting src/allgather_int.c


In [18]:
!mpicc -pedantic -Wall -o bin/allgather_int src/allgather_int.c
!mpiexec -n 4 bin/allgather_int

rank 0,sendbuf 0, recv_data 0123
rank 1,sendbuf 1, recv_data 0123
rank 2,sendbuf 2, recv_data 0123
rank 3,sendbuf 3, recv_data 0123


## Scatter

In [19]:
%%writefile src/scatter_int.c
#include<mpi.h>
#include<stdio.h>
#include<string.h>
#include<stdlib.h>
int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int send_data[4];
    if (rank==0){
        send_data[0]=4;
        send_data[1]=3;
        send_data[2]=2;
        send_data[3]=1;
    }
    int send_count=1;
    int recv_data[1];
    int recv_count=1;
    int root=0;
    
    MPI_Scatter(send_data, send_count, MPI_INT, recv_data, 
                recv_count, MPI_INT, root, MPI_COMM_WORLD);
    printf("rank %d, send_data %d, recv_data %d\n",rank,*(send_data+1),*recv_data);
    
    
    MPI_Finalize();
    return 0;
}

Overwriting src/scatter_int.c


In [20]:
!mpicc -pedantic -Wall -o bin/scatter_int src/scatter_int.c
!mpiexec -n 4 bin/scatter_int

rank 0, send_data 3, recv_data 4
rank 1, send_data 22029, recv_data 3
rank 2, send_data 21893, recv_data 2
rank 3, send_data 21874, recv_data 1


In [21]:
%%writefile src/scatter_char.c
#include<mpi.h>
#include<stdio.h>
#include<string.h>
#include<stdlib.h>
int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    char send_data[5];
    if (rank==0){
        send_data[0]='A';
        send_data[1]='B';
        send_data[2]='C';
        send_data[3]='D';
    }
    int send_count=1;
    char recv_data[2];
    recv_data[1]='\0';
    int recv_count=1;
    int root=0;
    
    MPI_Scatter(send_data, send_count, MPI_CHAR, recv_data, 
                recv_count, MPI_CHAR, root, MPI_COMM_WORLD);
    printf("rank %d, send_data %s, recv_data %s\n",rank,send_data,recv_data);
    
    
    MPI_Finalize();
    return 0;
}

Overwriting src/scatter_char.c


In [22]:
!mpicc -pedantic -Wall -o bin/scatter_char src/scatter_char.c
!mpiexec -n 4 bin/scatter_char

rank 0, send_data ABCD, recv_data A
rank 2, send_data S�, recv_data C
rank 1, send_data ^�, recv_data B
rank 3, send_data ��, recv_data D


# MPI Communicators

In [23]:
%%writefile src/cart_create.c
#include<mpi.h>
#include<stdio.h>
#define DIM 2

int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int dim_sizes[DIM]={4,size/4};
    int wrap_around[DIM]={0,1};
    MPI_Comm grid_comm;
    int reorder=1;
    MPI_Cart_create(MPI_COMM_WORLD, DIM, dim_sizes, wrap_around, reorder, &grid_comm);
    int grank;
    MPI_Comm_rank(grid_comm, &grank);
    int coordinates[DIM];
    MPI_Cart_coords(grid_comm,grank, DIM,coordinates);
    int grid_rank;
    MPI_Cart_rank(grid_comm,coordinates,&grid_rank);
    
    int free_coords[DIM]={0,1};
    MPI_Comm row_comm;
    MPI_Cart_sub(grid_comm, free_coords,&row_comm);
    int row_size, row_rank, row_cart_rank;
    MPI_Comm_rank(row_comm, &row_rank);
    MPI_Comm_size(row_comm, &row_size);
    MPI_Cart_rank(row_comm,coordinates,&row_cart_rank);

    
    //printf("rank: %2d; grank: %2d; [%d,%2d]; grid_rank: %2d; row_rank: %2d; row_size %2d\n",rank, grank,coordinates[0],coordinates[1], grid_rank,row_rank,row_size);
    //printf("===========\n");
    int rank_source, rank_dest;
    int direction=0;
    int displacement=1;
    MPI_Cart_shift(row_comm, direction, displacement, &rank_source,&rank_dest);
    printf("rank: %2d; grank: %2d; [%d,%2d]; grid_rank: %2d; row_rank: %2d; row_size %2d; rank_source %2d; rank_dest %2d; row_cart_rank %2d\n",
           rank, grank,coordinates[0],coordinates[1], grid_rank,row_rank,row_size, rank_source, rank_dest, row_cart_rank);   
    MPI_Finalize();
    return 0;
}

Overwriting src/cart_create.c


In [24]:
!mpicc -pedantic -Wall -o bin/cart_create src/cart_create.c
!mpiexec -n 20 --use-hwthread-cpus --oversubscribe bin/cart_create

rank:  0; grank:  0; [0, 0]; grid_rank:  0; row_rank:  0; row_size  5; rank_source  4; rank_dest  1; row_cart_rank  0
rank:  4; grank:  4; [0, 4]; grid_rank:  4; row_rank:  4; row_size  5; rank_source  3; rank_dest  0; row_cart_rank  0
rank: 12; grank: 12; [2, 2]; grid_rank: 12; row_rank:  2; row_size  5; rank_source  1; rank_dest  3; row_cart_rank  2
rank:  1; grank:  1; [0, 1]; grid_rank:  1; row_rank:  1; row_size  5; rank_source  0; rank_dest  2; row_cart_rank  0
rank:  3; grank:  3; [0, 3]; grid_rank:  3; row_rank:  3; row_size  5; rank_source  2; rank_dest  4; row_cart_rank  0
rank:  9; grank:  9; [1, 4]; grid_rank:  9; row_rank:  4; row_size  5; rank_source  3; rank_dest  0; row_cart_rank  1
rank: 11; grank: 11; [2, 1]; grid_rank: 11; row_rank:  1; row_size  5; rank_source  0; rank_dest  2; row_cart_rank  2
rank: 16; grank: 16; [3, 1]; grid_rank: 16; row_rank:  1; row_size  5; rank_source  0; rank_dest  2; row_cart_rank  3
rank: 17; grank: 17; [3, 2]; grid_rank: 17; row_rank:  2

# TODOs:
## `MPI_Cart_create, MPI_Cart_coords, MPI_Cart_sub, MPI_Cart_shift, MPI_Comm_rank`

# Bonus
## Assert:

In [25]:
%%writefile src/mpi_assert.c
#include<mpi.h>
#include<stdio.h>
#include<assert.h>
int main(int argc, char** argv){
    int rank, size;
    MPI_Init(&argc,&argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    assert(rank<size);
    MPI_Finalize();
    return 0;
}

Overwriting src/mpi_assert.c


In [26]:
!mpicc -pedantic -Wall -o bin/mpi_assert src/mpi_assert.c
!mpiexec -n 4 bin/mpi_assert